<a href="https://colab.research.google.com/github/rishabhranawat/prm-explorations/blob/main/Train_RM_PRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install -r /content/drive/MyDrive/colab/requirements.txt

In [ ]:
from datasets import load_dataset
from trl import PRMConfig, PRMTrainer
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score, f1_score
import wandb

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True' # Set early, before importing pytorch.

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "Qwen/Qwen2.5-0.5B",
    num_labels=2, id2label={0: "LABEL_0", 1: "LABEL_1"},
    label2id={"LABEL_0": 0, "LABEL_1": 1}
)

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Some weights of Qwen2ForTokenClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
)

peft_model = get_peft_model(model, peft_config)

In [ ]:
splits = ["train", "test"]
train_dataset, test_dataset = load_dataset("trl-lib/prm800k", split=splits)

README.md:   0%|          | 0.00/472 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.99M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/207k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/37482 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3695 [00:00<?, ? examples/s]

In [ ]:
training_args = PRMConfig(
    output_dir="/content/drive/MyDrive/qwen2-toy-rm-peft/v1", # updated name to reflect the use of PEFT
    logging_steps=10,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=2, # Reduces batch size to free up space.
    learning_rate=0.001,
    num_train_epochs=3,
    fp16=True,  # Enable mixed precision training
    gradient_checkpointing=True,  # Enable gradient checkpointing
)

# Initialize wandb
wandb.init(
    project="qwen-peft-reward-model",
    name="qwen-peft-rm-training-run",
    config=training_args.__dict__
)

# 6. Create trainer
trainer = PRMTrainer(
    model=peft_model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=test_dataset)

# 7. Train!
training_history = trainer.train()

# log the final training history to wandb.
wandb.log({
    "training_metrics": training_history.metrics,
})

# Finish wandb run
wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Tokenizing train dataset:   0%|          | 0/3695 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,49.874300
20,7.168800
30,4.573400
40,3.865900
50,4.116500
60,4.354000
70,3.686300
80,3.553800
90,3.145700
100,3.071100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,█▁▂▁▂▂▄▁▃▁▁▂▂▂▁▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▃▂▂▁▂▁▁▂▁
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▅▄▄▄▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,1.6839947385349032e+16
train/epoch,3
train/global_step,693
train/grad_norm,65.07935
train/learning_rate,2e-05
train/loss,1.9988


In [ ]:
# import gc
# import torch
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
def evaluate_stepwise_classification(model, tokenizer, examples, separator="\n"):
    pipe = pipeline("token-classification", model=model, tokenizer=tokenizer)
    all_preds = []
    all_labels = []
    all_results = []
    for example in examples:
        for idx in range(1, len(example["completions"]) + 1):
            steps = example["completions"][0:idx]
            text = separator.join((example["prompt"], *steps)) + separator
            pred_entity = pipe(text)[-1]["entity"] # get the last entity
            pred = {"LABEL_0": False, "LABEL_1": True}[pred_entity]
            label = example["labels"][idx - 1]
            all_preds.append(pred)
            all_labels.append(label)
            all_results.append({
               "prompt": example["prompt"],
               "steps_so_far": steps,
               "predicted": pred,
               "label": label
            })

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    return accuracy, f1

In [ ]:
base_model = AutoModelForTokenClassification.from_pretrained("gpt2", num_labels=2)
base_tokenizer = AutoTokenizer.from_pretrained("gpt2", max_length=1024, truncation=True)
base_tokenizer.pad_token = base_tokenizer.eos_token

tuned_model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/gpt2-toy-rm/v1/checkpoint-3900/",
                                                              num_labels=2)

Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print('Base GPT2 Performace Accuracy and F1 Score: ',
evaluate_stepwise_classification(
    base_model, base_tokenizer, test_dataset_filtered, separator="\n")
)

Device set to use cuda:0


Base GPT2 Performace Accuracy and F1 Score:  (0.13286163522012578, 0.017809439002671415)


In [ ]:
print('Tuned GPT2 Reward Model Performace Accuracy and F1 Score:',
evaluate_stepwise_classification(
    tuned_model, base_tokenizer, test_dataset_filtered, separator="\n")
)

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Tuned GPT2 Reward Model Performace Accuracy and F1 Score: (0.5480096919349255, 0.6836571539318765)


In [ ]:
from datasets import load_dataset
from transformers import pipeline

pipe = pipeline("token-classification", model=tuned_model, tokenizer=base_tokenizer)
example = {'prompt': 'Three pencils and a jumbo eraser cost $\\$1.24$. Five pencils and a jumbo eraser cost $\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?',
 'completions': ["Let's call the price of a pencil p and the price of a jumbo eraser e. Then we can write two equations.",
  'We have $3p+e=1.24$ and $5p+e=1.82$.',
  "To solve this system, let's subtract the first equation from the second equation. This will eliminate e.",
  '$5p+e-3p-e=1.82-1.24$.',
  'This simplifies to $2p=0.58$. So $p=0.29$.',
  'So the price of a pencil is 29 cents.\n\n# Answer\n\n29'],
 'labels': [True, True, True, True, True, True]}


separator = "\n"  # It's important to use the same separator as the one used during training

for idx in range(1, len(example["completions"]) + 1):
    steps = example["completions"][0:idx]
    text = separator.join((example["prompt"], *steps)) + separator  # Add a separator between the prompt and each steps
    pred_entity = pipe(text)[-1]["entity"]
    pred = {"LABEL_0": False, "LABEL_1": True}[pred_entity]
    label = example["labels"][idx - 1]
    print(f"Step {idx}\tPredicted: {pred} \tLabel: {label}")

Device set to use cuda:0


Step 1	Predicted: True 	Label: True
Step 2	Predicted: True 	Label: True
Step 3	Predicted: False 	Label: True
Step 4	Predicted: False 	Label: True
Step 5	Predicted: True 	Label: True
Step 6	Predicted: True 	Label: True


In [ ]:
evaluate_stepwise_classification(
    tuned_model,
    base_tokenizer,
    [test_dataset_filtered[8]]
)

Device set to use cuda:0


Three pencils and a jumbo eraser cost $\$1.24$. Five pencils and a jumbo eraser cost $\$1.82$. No prices include tax. In cents, what is the cost of a pencil?
[{'entity': 'LABEL_0', 'score': 0.8089562, 'index': 0, 'word': 'Three', 'start': 0, 'end': 5}, {'entity': 'LABEL_0', 'score': 0.65270364, 'index': 1, 'word': 'Ġpencil', 'start': 5, 'end': 12}, {'entity': 'LABEL_0', 'score': 0.8416726, 'index': 2, 'word': 's', 'start': 12, 'end': 13}, {'entity': 'LABEL_0', 'score': 0.8445638, 'index': 3, 'word': 'Ġand', 'start': 13, 'end': 17}, {'entity': 'LABEL_0', 'score': 0.705636, 'index': 4, 'word': 'Ġa', 'start': 17, 'end': 19}, {'entity': 'LABEL_1', 'score': 0.5172407, 'index': 5, 'word': 'Ġj', 'start': 19, 'end': 21}, {'entity': 'LABEL_0', 'score': 0.8438855, 'index': 6, 'word': 'umbo', 'start': 21, 'end': 25}, {'entity': 'LABEL_1', 'score': 0.51767355, 'index': 7, 'word': 'Ġer', 'start': 25, 'end': 28}, {'entity': 'LABEL_1', 'score': 0.54202014, 'index': 8, 'word': 'aser', 'start': 28, 'en

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(nan, 0.0)